#### Testing deconvolution

In [23]:
import RedLionfishDeconv as RL
from skimage.io import imread ,imsave
psf_path = "C:\\Users\\Pradeep\\Downloads\\PSF_RFI_8bit.tif"
img_path = "C:\\Users\\Pradeep\\Downloads\\gendata_psfconv_poiss_large.tif"

img = imread(img_path)
psf = imread(psf_path)

final = RL.doRLDeconvolutionFromNpArrays(data_np = img,psf_np = psf,method='gpu',useBlockAlgorithm=True)
imsave("D:\\decon_block.tif",final)

In [24]:
final2 = RL.doRLDeconvolutionFromNpArrays(data_np = img,psf_np = psf,method='gpu',useBlockAlgorithm=False)
imsave("D:\\decon_no_block.tif",final2)

ERROR:root:Shape (60, 1026, 1544) is too large for OpenCL device shape limits [1024, 1024, 64]


In [ ]:
#Get deskewing data and try it as a workflow

In [14]:
from napari_workflows import Workflow
import pyclesperanto_prototype as cle
image_seg_workflow = Workflow()
from skimage.io import imread

img = imread("C:\\Users\\deepu\\Desktop\\RBC_lattice_anisotropic.tif")

voxel_size_x_in_microns = 0.145
voxel_size_y_in_microns = 0.145
voxel_size_z_in_microns = 0.3
deskewing_angle_in_degrees = 30

input_arg = "input"
image_seg_workflow.set("deskewing", cle.deskew_y, input_arg, angle_in_degrees = deskewing_angle_in_degrees,
                    voxel_size_x = voxel_size_x_in_microns, voxel_size_y= voxel_size_y_in_microns,
                    voxel_size_z = voxel_size_z_in_microns)

image_seg_workflow.set("median", cle.median_sphere,"deskewing",radius_x = 2, radius_y = 2, radius_z = 2)

image_seg_workflow.set("binarisation", cle.threshold,"median",constant =1000)

image_seg_workflow.set("labeling", cle.connected_components_labeling_box,"binarisation")

print(str(image_seg_workflow))

Workflow:
deskewing <- (<function deskew_y at 0x000001A8DFECE1F0>, 'input', None, 30, 0.145, 0.145, 0.3, 1)
median <- (<function median_sphere at 0x000001A8DFD8E160>, 'deskewing', None, 2, 2, 2)
binarisation <- (<function greater_constant at 0x000001A8DFD725E0>, 'median', None, 1000)
labeling <- (<function connected_components_labeling_box at 0x000001A8DFE285E0>, 'binarisation')



In [2]:
from napari_workflows import Workflow
from napari_workflows._io_yaml_v1 import load_workflow, save_workflow

ImportError: cannot import name 'Viewer' from 'napari' (unknown location)

In [65]:
for key in image_seg_workflow._tasks.keys():
    for task in image_seg_workflow._tasks[key]:
        if hasattr(task, '__call__'): # if function
            print(task)
            #validate_task_install(task)
            #print(task.__module__)
            

<function deskew_y at 0x000001A8DFECE1F0>
<function median_sphere at 0x000001A8DFD8E160>
<function greater_constant at 0x000001A8DFD725E0>
<function connected_components_labeling_box at 0x000001A8DFE285E0>


In [4]:
import pyclesperanto_prototype as cle
from skimage.io import imread

img = imread("../sample_data/RBC_lattice.tif")

voxel_size_x_in_microns = 0.1449922
voxel_size_y_in_microns = 0.1449922
voxel_size_z_in_microns = 0.3

deskewing_angle_in_degrees = 30

print("Shape of original image", img.shape )

#Perform deskewing on rbc image  
deskewed = cle.deskew_y(img, 
                        angle_in_degrees=deskewing_angle_in_degrees, 
                        voxel_size_x=voxel_size_x_in_microns, 
                        voxel_size_y=voxel_size_y_in_microns, 
                        voxel_size_z=voxel_size_z_in_microns)

print("Shape of deskewed image",deskewed.shape)

Shape of original image (150, 118, 209)


Shape of deskewed image (59, 413, 209)


c:\Users\deepu\Anaconda3\envs\napari_lattice\lib\site-packages\napari_tools_menu\__init__.py:194: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


<Image layer 'deskewed' at 0x270362f9820>

In [7]:
from numba import njit, prange
import numpy as np

@njit(parallel=True)
def opm_deskew(data,theta,distance,pixel_size):
    """
    Perform parallelized orthogonal interpolation into a uniform pixel size grid.
    
    :param data: ndarray
        image stack of uniformly spaced OPM planes
    :param theta: float 
        angle relative to coverslip
    :param distance: float 
        step between image planes along coverslip
    :param pizel_size: float 
        in-plane camera pixel size in OPM coordinates
    :return output: ndarray
        image stack of deskewed OPM planes on uniform grid
    """

    # unwrap parameters 
    [num_images,ny,nx]=data.shape     # (pixels)

    # change step size from physical space (nm) to camera space (pixels)
    pixel_step = distance/pixel_size    # (pixels)

    # calculate the number of pixels scanned during stage scan 
    scan_end = num_images * pixel_step  # (pixels)

    # calculate properties for final image
    final_ny = np.int64(np.ceil(scan_end+ny*np.cos(theta*np.pi/180))) # (pixels)
    final_nz = np.int64(np.ceil(ny*np.sin(theta*np.pi/180)))          # (pixels)
    final_nx = np.int64(nx)                                           # (pixels)

    # create final image
    output = np.zeros((final_nz, final_ny, final_nx),dtype=np.float32)  # (time, pixels,pixels,pixels - data is float32)

    # precalculate trig functions for scan angle
    tantheta = np.float32(np.tan(theta * np.pi/180)) # (float32)
    sintheta = np.float32(np.sin(theta * np.pi/180)) # (float32)
    costheta = np.float32(np.cos(theta * np.pi/180)) # (float32)

    # perform orthogonal interpolation

    # loop through output z planes
    # defined as parallel loop in numba
    # http://numba.pydata.org/numba-doc/latest/user/parallel.html#numba-parallel
    for z in prange(0,final_nz):
        # calculate range of output y pixels to populate
        y_range_min=np.minimum(0,np.int64(np.floor(np.float32(z)/tantheta)))
        y_range_max=np.maximum(final_ny,np.int64(np.ceil(scan_end+np.float32(z)/tantheta+1)))

        # loop through final y pixels
        # defined as parallel loop in numba
        # http://numba.pydata.org/numba-doc/latest/user/parallel.html#numba-parallel
        for y in prange(y_range_min,y_range_max):

            # find the virtual tilted plane that intersects the interpolated plane 
            virtual_plane = y - z/tantheta

            # find raw data planes that surround the virtual plane
            plane_before = np.int64(np.floor(virtual_plane/pixel_step))
            plane_after = np.int64(plane_before+1)

            # continue if raw data planes are within the data range
            if ((plane_before>=0) and (plane_after<num_images)):
                
                # find distance of a point on the  interpolated plane to plane_before and plane_after
                l_before = virtual_plane - plane_before * pixel_step
                l_after = pixel_step - l_before
                
                # determine location of a point along the interpolated plane
                za = z/sintheta
                virtual_pos_before = za + l_before*costheta
                virtual_pos_after = za - l_after*costheta

                # determine nearest data points to interpoloated point in raw data
                pos_before = np.int64(np.floor(virtual_pos_before))
                pos_after = np.int64(np.floor(virtual_pos_after))

                # continue if within data bounds
                if ((pos_before>=0) and (pos_after >= 0) and (pos_before<ny-1) and (pos_after<ny-1)):
                    
                    # determine points surrounding interpolated point on the virtual plane 
                    dz_before = virtual_pos_before - pos_before
                    dz_after = virtual_pos_after - pos_after

                    # compute final image plane using orthogonal interpolation
                    output[z,y,:] = (l_before * dz_after * data[plane_after,pos_after+1,:] +
                                    l_before * (1-dz_after) * data[plane_after,pos_after,:] +
                                    l_after * dz_before * data[plane_before,pos_before+1,:] +
                                    l_after * (1-dz_before) * data[plane_before,pos_before,:]) /pixel_step


    # return output
    return output

In [12]:
deskew_opm = opm_deskew(img,30.0,0.3,0.1449922)
deskew_opm = np.flip(deskew_opm,axis=0) #coverslip rotation

print(deskew)

import napari 

viewer = napari.Viewer() 

viewer.add_image(img)
viewer.add_image(deskewed)
viewer.add_image(deskew_opm)

c:\Users\deepu\Anaconda3\envs\napari_lattice\lib\site-packages\napari_tools_menu\__init__.py:194: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


<Image layer 'deskew_opm' at 0x270427f7790>

c:\Users\deepu\Anaconda3\envs\napari_lattice\lib\site-packages\napari\layers\image\image.py:839: RuntimeWarning: divide by zero encountered in true_divide
  raw_zoom_factor = np.divide(
c:\Users\deepu\Anaconda3\envs\napari_lattice\lib\site-packages\napari\layers\image\image.py:847: RuntimeWarning: divide by zero encountered in true_divide
  zoom_factor = tuple(new_shape / image.shape[:2])
